In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@vvdatabricksstorage.dfs.core.windows.net/orders")

display(df)

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")

In [0]:
df = df.drop("rescued_data")
display(df)

In [0]:
df = df.withColumn("order_date", to_timestamp(col('order_date')))
display(df)

In [0]:
df = df.withColumn("year", year(col('order_date')))
display(df)

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("row_number", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

### **Classes**

In [0]:
class windows:
  def dense_rank(self, df):
    return df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
  
  def rank(self, df):
    return df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))  
  
  def row_number(self, df):  
    return df.withColumn("row_number", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))


In [0]:
df_new = df

In [0]:
obj = windows()
df_new = obj.dense_rank(df_new)
df_new = obj.rank(df_new)
df_new = obj.row_number(df_new)
display(df_new)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@vvdatabricksstorage.dfs.core.windows.net/orders")

In [0]:
%sql
create table if not exists databricks_cata.silver.orders_silver
using delta
location "abfss://silver@vvdatabricksstorage.dfs.core.windows.net/orders"